# Implementing Iteration

## Agenda

1. Review: Iteration
2. Details: *iterables*, *iterators*, `iter`, and `next`
3. Implementing iterators with classes
4. Implementing iterators with *generators* and `yield`

## 1. Iteration

*Iteration* simply refers to the process of accessing — one by one — the items stored in some container. The order of the items, and whether or not the iteration is comprehensive, depends on the container.

In Python, we typically perform iteration using the `for` loop.

In [1]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [2]:
for c in 'CS 331':
    print(c)

C
S
 
3
3
1


In [3]:
for e in [3.14, 'hello', True]:
  print(e)

3.14
hello
True


In [4]:
for n in {5, 6, 7, 5, 3, 0, 9}:
  print(n)

0
3
5
6
7
9


In [5]:
for k,v in {'a': 'aardvark', 'b': 'bear', 'c': 'cat'}.items():
    print(f'{k} -> {v}')

a -> aardvark
b -> bear
c -> cat


## 2. *iterables*, *iterators*, `iter`, and `next`

We can iterate over anything that is *iterable*. Intuitively, if something can be used as the source of items in a `for` loop, it is iterable.

But how does a `for` loop really work?

An *iterator* is an API that a class can conform to.

In [6]:
l = ['iteration', 'is', 'really', 'very', 'convenient']

In [17]:
it = iter(l)

In [15]:
type(l), type(it)

(list, list_iterator)

In [18]:
next(it) # next element from the underlying iterable

'iteration'

However, iterators can only be used once, and new ones must be created to use them again.

For-loops extract an iterator from an object (an iterable) and run an infinite loop calling the `next` method in each iteration, only breaking when the iterator raises a `StopIteration` exception.

In [20]:
# putting it all together
it = iter(l)
while True:
    try:
        x = next(it)
        print(x)
    except StopIteration:
        break

iteration
is
really
very
convenient


In [21]:
# iter & next are based on the __iter__ and __next__ special methods
it = l.__iter__()
while True:
    try:
        x = it.__next__()
        print(x)
    except StopIteration:
        break

iteration
is
really
very
convenient


## 3. Implementing iterators with classes

In [22]:
class MyIterator:
    def __init__(self, max):
        self.max = max
        self.curr = 0
        
    # the following methods are required for iterator objects

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.curr < self.max:
            ret = self.curr
            self.curr += 1
            return ret
        else:
            raise StopIteration()

In [35]:
it = MyIterator(10)

In [46]:
next(it)

StopIteration: 

In [47]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


In [48]:
it = MyIterator(10)
for i in it:
    print(i)

0
1
2
3
4
5
6
7
8
9


An iterator is a *one time use object*! I.e., once we've used it to iterate over elements we cannot typically reset or "rewind" iteration. 

In [49]:
it = MyIterator(10)

In [51]:
# try running this cell multiple times
for i in it:
  print(i)

Iterable objects that can be traversed repeatedly return fresh iterators for each traversal.

In [52]:
# what is happening each time we iterate over `l`, below?

l = list(range(10))
total = 0
for _ in range(10):
    for x in l:
        total += x
total

450

In [53]:
l = list(range(10))
total = 0
for _ in range(10):
    it = iter(l) # we obtain and "use up" a new iterator each loop!
    while True:
        try:
            total += next(it)
        except StopIteration:
            break
total

450

Note that since iterating ove an object requires creating an iterator, some overhead is involved.

For a container type, we need to implement an `__iter__` method that returns an iterator.

In [54]:
import numpy as np

class ArrayListIterator:
    def __init__(self, array_list):
        self.array_list = array_list # the underlying iterable
        self.curr_idx = 0 # our position in it

    def __iter__(self):
        return self

    def __next__(self):
        if self.curr_idx < self.array_list.size:
            ret = self.array_list.data[self.curr_idx]
            self.curr_idx += 1
            return ret
        else:
            raise StopIteration()


class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(2 * len(self.data), dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        return ArrayListIterator(self) # create and return an iterator

In [55]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [56]:
it = iter(l)

In [57]:
type(it)

__main__.ArrayListIterator

In [68]:
next(it)

StopIteration: 

In [69]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


## 4. Implementing iterators with generators

What's a "generator"? A one-time lazily created sequence.

Objects in a generator are *lazily* created: that is, they are only created as they are needed.

In [70]:
l = [2**x for x in range(10)]
g = (2**x for x in range(10))

In [71]:
type(l), type(g)

(list, generator)

In [74]:
# try running this cell repeatedly
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [76]:
# try running this cell repeatedly
for x in g:
    print(x)

In [77]:
dir(g)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

Note: Generators implement the iterator API! (`__iter__` and `__next__`)

In [78]:
g = (2**x for x in range(10))

In [79]:
g is iter(g)

True

In [90]:
next(g)

StopIteration: 

In [91]:
%timeit -n 1000 [2**x for x in range(1000)]

588 µs ± 50.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [92]:
%timeit -n 1000 (2**x for x in range(1000))

901 ns ± 416 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [93]:
g = (2**x for x in range(100))

In [94]:
g[100]

TypeError: 'generator' object is not subscriptable

In [96]:
# try running this cell repeatedly
sum(g)

0

A *generator expression* syntactically resembles a list comprehension, and is similar in that it evaluates to an iterable sequence of values. However, a generator does not represent a fully fleshed out collection of values; instead, values are returned only as they are required through the iteration API (i.e., `next`) --- we refer to this as *lazy evaluation*. 

This makes a generator more efficient than a list (since we don't need to keep all values in the sequence around), but generators can't replace lists in all scenarios (e.g., when we need to jump around in the sequence or revisit values).

### Creating generator functions: `yield`

The presence of the `yield` keyword completely changes the nature of a function: it now becomes a *generator* function. `yield` pauses the execution of the generator function, and when the function is called again, execution resumes from its previous state.

In [97]:
def foo():
    yield

In [101]:
foo()

<generator object foo at 0x72dfed41ac00>

In [102]:
type(foo())

generator

In [103]:
def foo():
    print('hello!')
    yield
    print('goodbye!')

In [105]:
foo()

<generator object foo at 0x72dfed41af10>

In [106]:
g = foo()

In [108]:
next(g)

goodbye!


StopIteration: 

In [109]:
def foo():
    yield 1
    yield 2
    yield 3

In [110]:
g = foo()

In [114]:
next(g)

StopIteration: 

In [115]:
def countdown(n):
    for x in range(n, 0, -1):
        yield x
    yield 'Blast off!'

In [116]:
for x in countdown(5):
    print(x)

5
4
3
2
1
Blast off!


In [117]:
list(countdown(10))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 'Blast off!']

A *generator function* is a function that contains one or more `yield` statements. When called, a generator function returns a generator object, which effectively allows us to incrementally execute the function using the iteration API. Each call to `next` on the generator will execute the function up to the next `yield` statement; if/when the function completes the generator will raise a `StopIteration` exception (just like an iterator).

### Generators as Data Structure Iterators

In [118]:
class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(len(self.data)*2, dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        for i in range(self.size):
            yield self.data[i]

In [119]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [120]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [123]:
# use the now functioning iterator to implement __repr__
class ArrayList(ArrayList):
    def __repr__(self):
        return '[' + ', '.join(repr(x) for x in self) + ']'

In [124]:
l = ArrayList()
for x in range(10):
    l.append(2**x)
l

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]